In [21]:
import clr
clr.AddReference("System")
clr.AddReference("QuantConnect.Algorithm")
clr.AddReference("QuantConnect.Indicators")
clr.AddReference("QuantConnect.Common")

from System import *
from QuantConnect import *
from QuantConnect.Algorithm import *
from QuantConnect.Indicators import *
import numpy as np
from datetime import timedelta
from sklearn.linear_model import LinearRegression

class OptionsTest(QCAlgorithm):
    
    def Initialize(self):
        self.SetCash(100000)
        

        self.SetStartDate(2014,6,30)
        self.SetEndDate(2014,7,1)
        
        # Adding  assets you'd like to see
        self.AddEquity("SPY", Resolution.Daily)
        self.AddEquity("XIV", Resolution.Daily)
        option = self.AddOption("SPY", Resolution.Daily)
        option.SetFilter(-2, 2, timedelta(0), timedelta(182))
        self.o_symbol=option.Symbol    


    def OnData(self, slice):
        d1=self.History('SPY',21,Resolution.Daily)
        d2=self.History('XIV',21,Resolution.Daily)
        index=[]
        for slice1,slice2 in zip(d1,d2):
            index.append(slice1.Time.date())
            data.append([np.float(slice1.Close),np.float(slice2.Close)])
        df=pd.DataFrame(data,columns=['SPY','XIV'],index=pd.Series(index))
        rr=df.diff().fillna(0)
        lr=LinearRegression()
        X=np.column_stack((rr['SPY'].values,rr['SPY'].values**2))
        y=rr['XIV'].values
        lr.fit(X,y)
        resid=y-lr.predict(X)
        resid=resid/resid.std()
        for kvp in slice.OptionChains:
            if kvp.Key != self.o_symbol:continue
            chain = kvp.Value
            contracts = sorted(sorted(chain,key = lambda x: abs(chain.Underlying.Price - x.Strike)),
                               key = lambda x: x.Expiry, reverse=True)
            if len(contracts) == 0: continue
            symbol = contracts[0].Symbol
            if resid[-1]>1:
                self.MarketOnCloseOrder(symbol, -1)
            if resid[-1]<-1:
                self.MarketOnCloseOrder(symbol,  1)

ModuleNotFoundError: No module named 'clr'

In [18]:
import numpy as np

class SimpleBreakoutExample(QCAlgorithm):

    def Initialize(self):
        # Set the cash for backtest
        self.SetCash(100000)
        
        # Start and end dates for backtest
        self.SetStartDate(2017,9,1)
        self.SetEndDate(2020,9,1)
        
        # Add asset
        self.symbol = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Lookback length for b/o (in days)
        self.lookback = 20
        
        # Upper/lower limit for lookback length
        self.ceiling, self.floor = 30, 10
        
        # Price offset for stop order
        self.initialStopRisk = 0.98
        self.trailingStopRisk = 0.9
        
        # Schedule function 20 minutes after every market open
        self.Schedule.On(self.DateRules.EveryDay(self.symbol), \
                        self.TimeRules.AfterMarketOpen(self.symbol, 20), \
                        Action(self.EveryMarketOpen))


    def OnData(self, data):
        # Plot security's price
        self.Plot("Data Chart", self.symbol, self.Securities[self.symbol].Close)

 
    def EveryMarketOpen(self):
        # Dynamically determine lookback length based on 30 day volatility change rate
        close = self.History(self.symbol, 31, Resolution.Daily)["close"]
        todayvol = np.std(close[1:31])
        yesterdayvol = np.std(close[0:30])
        deltavol = (todayvol - yesterdayvol) / todayvol
        self.lookback = round(self.lookback * (1 + deltavol))
        
        # Account for upper/lower limit of lockback length
        if self.lookback > self.ceiling:
            self.lookback = self.ceiling
        elif self.lookback < self.floor:
            self.lookback = self.floor
        
        # List of daily highs
        self.high = self.History(self.symbol, self.lookback, Resolution.Daily)["high"]
        
        # Buy in case of breakout
        if not self.Securities[self.symbol].Invested and \
                self.Securities[self.symbol].Close >= max(self.high[:-1]):
            self.SetHoldings(self.symbol, 1)
            self.breakoutlvl = max(self.high[:-1])
            self.highestPrice = self.breakoutlvl
        
        
        # Create trailing stop loss if invested 
        if self.Securities[self.symbol].Invested:
            
            # If no order exists, send stop-loss
            if not self.Transactions.GetOpenOrders(self.symbol):
                self.stopMarketTicket = self.StopMarketOrder(self.symbol, \
                                        -self.Portfolio[self.symbol].Quantity, \
                                        self.initialStopRisk * self.breakoutlvl)
            
            # Check if the asset's price is higher than highestPrice & trailing stop price not below initial stop price
            if self.Securities[self.symbol].Close > self.highestPrice and \
                    self.initialStopRisk * self.breakoutlvl < self.Securities[self.symbol].Close * self.trailingStopRisk:
                # Save the new high to highestPrice
                self.highestPrice = self.Securities[self.symbol].Close
                # Update the stop price
                updateFields = UpdateOrderFields()
                updateFields.StopPrice = self.Securities[self.symbol].Close * self.trailingStopRisk
                self.stopMarketTicket.Update(updateFields)
                
                # Print the new stop price with Debug()
                self.Debug(updateFields.StopPrice)
            
            # Plot trailing stop's price
            self.Plot("Data Chart", "Stop Price", self.stopMarketTicket.Get(OrderField.StopPrice))

NameError: name 'QCAlgorithm' is not defined